In [1]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

data210802


In [3]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [ ]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

In [3]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

In [1]:
import pandas as pd
import numpy as np
import glob
from sklearn.preprocessing import LabelEncoder

In [2]:
train_goods = pd.concat([
    pd.read_csv('./训练集/traindata_goodsid/part-00000', header=None, names=['goods_id', 'cat_id', 'brandsn']),
    pd.read_csv('./训练集/traindata_goodsid/part-00001', header=None, names=['goods_id', 'cat_id', 'brandsn']),
    pd.read_csv('./训练集/traindata_goodsid/part-00002', header=None, names=['goods_id', 'cat_id', 'brandsn'])
], axis=0)

train_user = pd.concat([
    pd.read_csv(x, header=None, names=['user_id', 'goods_id', 'is_clk', 'is_like', 'is_addcart', 'is_order', 'expose_start_time', 'dt'], nrows=None)
    for x in glob.glob('./训练集/traindata_user/part*')
], axis=0)


In [3]:
import math
train_user['date'] = train_user['dt'].apply(lambda x : int(str(x)[6:]))
train_user['date'] = train_user['date'].apply(lambda x:math.exp((30-x)/7*math.log(0.5,math.e)))
train_user

,user_id,goods_id,is_clk,is_like,is_addcart,is_order,expose_start_time,dt,date
0,531efbdc210c51ca1f5bc2a16ce9f5c0,e260db75426ce304e38cd7b7b5e33b66,1,0,1,0,2023-02-10 11:19:58,20230210,0.138011
1,cf83335e06b39155eb052d26b18922ee,b293ad31ae6f710f06beba07c9eee017,1,0,2,0,2023-02-10 01:39:56,20230210,0.138011
2,ab3db52b356db7abf631db13e6cc601a,4eff89e7ec5f58d488f76b0e8564609e,1,0,0,0,2023-02-10 22:30:47,20230210,0.138011
3,ab3db52b356db7abf631db13e6cc601a,11cd62b9957d7cf6792d5a3be7ebb361,1,0,0,0,2023-02-10 08:46:52,20230210,0.138011
4,61531eac347f8705971c46099494042f,342499939bb7f4864cae1800b3d4b3c3,1,0,0,0,2023-02-10 20:59:25,20230210,0.138011
...,...,...,...,...,...,...,...,...,...
140130,5d9fb9095692a437df356bcf526a7892,f5b446a9148a625d084f5f1ba6cd9f96,1,0,0,0,2023-02-23 21:06:37,20230223,0.500000
140131,1f28fcb4e9d196fff1bee2c8a918f035,67847f86694f77025febc3bcd07222f6,1,0,0,0,2023-02-23 11:51:53,20230223,0.500000
140132,b89b7dab6632d1341458f53122f540d2,85bb34c5adcb5703e432373cdbfbd2c0,1,0,0,0,2023-02-23 14:33:04,20230223,0.500000
140133,b89b7dab6632d1341458f53122f540d2,b195478c12d4d1f60873633177406f7d,1,0,0,0,2023-02-23 14:33:04,20230223,0.500000


In [4]:
testa_goods = pd.concat([
    pd.read_csv('./测试集a/predict_goods_id/part-00000', header=None, names=['goods_id']),
    pd.read_csv('./测试集a/predict_goods_id/part-00001', header=None, names=['goods_id']),
], axis=0)

testa_user = pd.read_excel('./测试集a/a榜需要预测的uid_5000.xlsx')

In [5]:
train_goods = train_goods[train_goods['goods_id'].isin(train_user['goods_id'])]


In [6]:
train_agg_feat = train_user.iloc[:].groupby(['user_id']).agg({
    'is_clk': ['sum'],
    'is_like': ['sum'],
    'is_addcart': ['sum'],
    'is_order': ['sum'],
})
train_agg_feat = train_agg_feat.reset_index()
train_agg_feat.columns = [
    'user_id',
    'is_clk_sum',
    'is_like_sum',
    'is_addcart_sum',
    'is_order_sum',
 ]
train_agg_feat = train_agg_feat[train_agg_feat['is_order_sum']!=0]
train_agg_feat

,user_id,is_clk_sum,is_like_sum,is_addcart_sum,is_order_sum
5,000c81287b218ec3e2245a33464992fe,343,0,14,2
10,0015421d3e9341bf4f1674f7dd4aa583,311,2,3,3
13,0016e9cf56c15284ac1ac8d2781f1463,159,6,5,4
19,0020de0bf2d10966dcc5604aa7057f2a,249,0,22,5
24,0023b4e1078f2a68442c0c32d5aec756,89,0,11,1
...,...,...,...,...,...
51590,fff7ea17b2a7f54e80791950b3cd0b0b,144,0,4,1
51597,fffd220636f9020f68abbe55626504e9,469,0,59,12
51599,fffedfaa8fd18e0b41d4756b1761b479,40,0,3,2
51600,ffff32fb6141c3391f49ff969adfa505,468,48,22,2


In [7]:
train_agg_feat.eval('clk_turn_rate = (is_order_sum/is_clk_sum)',inplace=True)
train_agg_feat.eval('like_turn_rate = (is_order_sum/is_like_sum)',inplace=True)
train_agg_feat.eval('addcart_turn_rate = (is_order_sum/is_addcart_sum)',inplace=True)
train_agg_feat = train_agg_feat.round({'clk_turn_rate': 2, 'like_turn_rate':2,'addcart_turn_rate':2})

train_agg_feat.replace(np.inf, 0,inplace=True) #替换inf为0

train_agg_feat['like_turn_rate'] = train_agg_feat['like_turn_rate']

In [8]:
train_agg_feat

,user_id,is_clk_sum,is_like_sum,is_addcart_sum,is_order_sum,clk_turn_rate,like_turn_rate,addcart_turn_rate
5,000c81287b218ec3e2245a33464992fe,343,0,14,2,0.01,0.00,0.14
10,0015421d3e9341bf4f1674f7dd4aa583,311,2,3,3,0.01,1.50,1.00
13,0016e9cf56c15284ac1ac8d2781f1463,159,6,5,4,0.03,0.67,0.80
19,0020de0bf2d10966dcc5604aa7057f2a,249,0,22,5,0.02,0.00,0.23
24,0023b4e1078f2a68442c0c32d5aec756,89,0,11,1,0.01,0.00,0.09
...,...,...,...,...,...,...,...,...
51590,fff7ea17b2a7f54e80791950b3cd0b0b,144,0,4,1,0.01,0.00,0.25
51597,fffd220636f9020f68abbe55626504e9,469,0,59,12,0.03,0.00,0.20
51599,fffedfaa8fd18e0b41d4756b1761b479,40,0,3,2,0.05,0.00,0.67
51600,ffff32fb6141c3391f49ff969adfa505,468,48,22,2,0.00,0.04,0.09


In [9]:
user_data = train_agg_feat.drop(['is_clk_sum','is_like_sum','is_addcart_sum'],axis =1)
user_data

,user_id,is_order_sum,clk_turn_rate,like_turn_rate,addcart_turn_rate
5,000c81287b218ec3e2245a33464992fe,2,0.01,0.00,0.14
10,0015421d3e9341bf4f1674f7dd4aa583,3,0.01,1.50,1.00
13,0016e9cf56c15284ac1ac8d2781f1463,4,0.03,0.67,0.80
19,0020de0bf2d10966dcc5604aa7057f2a,5,0.02,0.00,0.23
24,0023b4e1078f2a68442c0c32d5aec756,1,0.01,0.00,0.09
...,...,...,...,...,...
51590,fff7ea17b2a7f54e80791950b3cd0b0b,1,0.01,0.00,0.25
51597,fffd220636f9020f68abbe55626504e9,12,0.03,0.00,0.20
51599,fffedfaa8fd18e0b41d4756b1761b479,2,0.05,0.00,0.67
51600,ffff32fb6141c3391f49ff969adfa505,2,0.00,0.04,0.09


In [10]:
np.mean(testa_user['user_id'].isin(user_data['user_id'])), np.mean(testa_goods['goods_id'].isin(train_goods['goods_id']))

(0.5168, 0.6660650426473211)

In [11]:

testa_user = testa_user[testa_user['user_id'].isin(user_data['user_id'])]
testa_goods = testa_goods[testa_goods['goods_id'].isin(train_goods['goods_id'])]
score = train_user.iloc[:].groupby(['user_id','goods_id','date']).agg({
    'is_clk':['sum'],
    'is_like':['sum'],
    'is_addcart':['sum'],
    'is_order': ['sum'],
})
score =score.reset_index()
score.columns = [
    'user_id',
    'goods_id',
    'date',
    'is_clk_sum',
    'is_like_sum',
    'is_addcart_sum',
    'is_order_sum',
 ]
score.eval('score=(is_clk_sum+is_like_sum*2.17+is_addcart_sum*5.19+is_order_sum*15.65)*date',inplace=True)
score = score[score['user_id'].isin(user_data['user_id'])&score['goods_id'].isin(train_goods['goods_id'])]

In [16]:
np.mean(testa_user['user_id'].isin(user_data['user_id'])), np.mean(testa_goods['goods_id'].isin(train_goods['goods_id']))

(1.0, 1.0)

In [12]:
user_data['clk_turn_rate'] = pd.cut(user_data['clk_turn_rate'],20,labels=False) # 使用cut进行等宽离散化
user_data['like_turn_rate'] =  pd.cut(user_data['like_turn_rate'],20,labels=False)
user_data['addcart_turn_rate'] =  pd.cut(user_data['addcart_turn_rate'],20,labels=False)
user_data

,user_id,is_order_sum,clk_turn_rate,like_turn_rate,addcart_turn_rate
5,000c81287b218ec3e2245a33464992fe,2,0,0,0
10,0015421d3e9341bf4f1674f7dd4aa583,3,0,0,1
13,0016e9cf56c15284ac1ac8d2781f1463,4,0,0,0
19,0020de0bf2d10966dcc5604aa7057f2a,5,0,0,0
24,0023b4e1078f2a68442c0c32d5aec756,1,0,0,0
...,...,...,...,...,...
51590,fff7ea17b2a7f54e80791950b3cd0b0b,1,0,0,0
51597,fffd220636f9020f68abbe55626504e9,12,0,0,0
51599,fffedfaa8fd18e0b41d4756b1761b479,2,0,0,0
51600,ffff32fb6141c3391f49ff969adfa505,2,0,0,0


In [13]:
train_goods

,goods_id,cat_id,brandsn
1,3401512d6e8599ef04cef828dfe7fc0f,0d0fd7c6e093f7b804fa0150b875b868,c60a61c10d2a17a26e11e2175ec47740
4,0c70bdad70b2ca5f8afaf06ec9c66304,9c5a77f5b39328c7e887f54be2ec1aca,c60a61c10d2a17a26e11e2175ec47740
5,1135f1793b9609f3aaa412c1aa596fa5,27964b9a28619992910ab94b9894fd55,c60a61c10d2a17a26e11e2175ec47740
7,7f05b082fa8d837f50943b4b47ebbe71,7d5cd18de5c6874db36b3aae26089831,647356ee6f6f19cc803e4ee345a2e328
8,cac9b70d613de53eade66981114f4902,9c37311816d6a325ded1da652bd13af0,1b1c5b8f64e4cc748698d3680f3adc67
...,...,...,...
1045258,b18bccbf6cd17438ea9d5ea360deda90,51d4658087da422d04101a9a0a285704,36b058818a9d5058a0ebf3f13080348f
1045266,4d4af789ef767f8f01cea63a03323a29,a666587afda6e89aec274a3657558a27,827e9c4b6b35341601983da8d9ea2137
1045309,7d0139968cb685253180918798773f06,e9b77eef7828123c52f170cd21a4e0e9,28da51e067840f763c8785c10765c02f
1045370,aa5ef4c3b6c0526e42bb7637685bf77c,29a3c8150ea0f1c0e21142471f6fc77e,1229827acd85e60cf1f94e8986bc9d7f


In [19]:
testa_user

,user_id
0,db55512e13c666a67dbfeae200bbb77e
1,6be82d1677c1eacfc615446329fa90b1
3,887343b7ef8d77baed6b2fdfc4b8f536
5,5cb2378cab17ed27dcbca54a248999a7
6,c37822c30576714979c146f01c7c5863
...,...
4991,c1f875cefb8e00ac868a98f0a8d912ff
4993,773d6d45c6dddbfb07675e5e2308dad4
4995,1aaea0961cac2a6b9f9a80dc6e54ff88
4996,0d0456f67f5a690c5b3623b832fdad32


In [14]:
testa_goods

,goods_id
0,73e4cff7f7dca90b90fba589709cde14
1,443d7b759f77e4191c9cd3ef1575befc
2,839e4333054d929bcf284fccc7cadba6
3,52411d011ff1c09e8767fc3a2be8708a
4,d0d40297acaf73fcf492550e969d89d1
...,...
684094,4572a7e238db3af0495ac276db25394a
684095,06196133e37467a7ec9dfdb3b5fb4bd0
684097,014c824410cf6cba025a7fe1f6a95dfb
684099,ad27805b7ca9fcf4726053ba5b33f0cf


In [15]:

max_tmp = np.max(np.array(score['score']))
min_tmp = np.min(np.array(score['score']))
if (max_tmp != min_tmp):
    score['score'] = score['score'].apply(lambda x: (x - min_tmp) / (max_tmp - min_tmp))
score = score.round({'score':2})
score

,user_id,goods_id,date,is_clk_sum,is_like_sum,is_addcart_sum,is_order_sum,score
297,000c81287b218ec3e2245a33464992fe,011d685ae5df37c641aec66f83cf57d1,0.084119,1,0,0,0,0.00
298,000c81287b218ec3e2245a33464992fe,018e8d671160b84fc3c1625178e0dcf6,0.084119,1,0,0,0,0.00
299,000c81287b218ec3e2245a33464992fe,02fba6bd3fe1e01db229ed963dc7d839,0.138011,1,0,0,0,0.00
300,000c81287b218ec3e2245a33464992fe,06e225e957ffb9bfdd0d86f875052759,0.371499,1,0,1,0,0.02
301,000c81287b218ec3e2245a33464992fe,099afe8720e4ef34c3c55c5e13c58645,0.069006,1,0,0,0,0.00
...,...,...,...,...,...,...,...,...
7587321,ffffb2aad8fd839b6f152ae9126a05ff,fd6907060fb1e080f5a996347d9d2020,0.185749,1,0,0,0,0.00
7587322,ffffb2aad8fd839b6f152ae9126a05ff,fef0e42ef8f326a7691053f4a4de000b,0.672950,1,0,0,0,0.01
7587323,ffffb2aad8fd839b6f152ae9126a05ff,fefb7378e47a48cf9ee25ebee5d478ff,0.092875,1,0,0,0,0.00
7587324,ffffb2aad8fd839b6f152ae9126a05ff,ff8b7d565556bcfced37258d69ce6459,0.672950,2,0,0,0,0.01


In [16]:
user_encode = LabelEncoder()
user_encode.fit(list(user_data['user_id']))

goods_encode = LabelEncoder()
goods_encode.fit(list(train_goods['goods_id']))

cat_encode = LabelEncoder()
cat_encode.fit(list(train_goods['cat_id']))

brandsn_encode = LabelEncoder()
brandsn_encode.fit(list(train_goods['brandsn']))

LabelEncoder()

In [17]:
user_data['user_id'] = user_encode.transform(user_data['user_id'])
train_goods['goods_id'] = goods_encode.transform(train_goods['goods_id'])
testa_user['user_id'] = user_encode.transform(testa_user['user_id'])
testa_goods['goods_id'] = goods_encode.transform(testa_goods['goods_id'])
score['user_id'] = user_encode.transform(score['user_id'])
score['goods_id']= goods_encode.transform(score['goods_id'])
train_goods['cat_id'] = cat_encode.transform(train_goods['cat_id'])
train_goods['brandsn'] = brandsn_encode.transform(train_goods['brandsn'])

In [18]:
import random
import numpy as np

class GoodsLen(object):
    def __init__(self,user_data,train_goods,score):
        #记录商品数据最大的id,cat,brand
        self.max_goods_id = 0
        self.max_goods_cat = 0
        self.max_goods_brand = 0
        # 得到商品数据
        self.goods_info = self.get_goods_info(train_goods)
        # 记录用户数据的最大ID
        self.max_usr_id = 0
        self.max_usr_clk = 0
        self.max_usr_like = 0
        self.max_usr_addcart = 0
        # 得到用户数据
        self.usr_info = self.get_usr_info(user_data)
        # 得到购买数据
        self.rating_info = self.get_rating_info(score)
        # 构建数据集 
        self.dataset = self.get_dataset(usr_info=self.usr_info,
                                        rating_info=self.rating_info,
                                        goods_info=self.goods_info)
        # 划分数据集，获得数据加载器
        self.train_dataset = self.dataset[:int(len(self.dataset)*0.9)]
        self.valid_dataset = self.dataset[int(len(self.dataset)*0.9):]
        print("##Total dataset instances: ", len(self.dataset))
        print("##GoodsLens dataset information: \nusr num: {}\n"
              "goods num: {}".format(len(self.usr_info),len(self.goods_info)))
    # 得到商品数据
    def get_goods_info(self, df):
        # 建立三个字典，分别存放商品所有信息
        goods_info={}
        
        # 按行读取数据并处理
        for index, row in df.iterrows():
            v_id = row['goods_id']
            v_cats = row['cat_id']
            v_brandsn = row['brandsn']
            goods_info[v_id] = {'goods_id': v_id,
                                'category': v_cats,
                                'brand': v_brandsn}
            self.max_goods_cat = max(self.max_goods_cat,v_cats)
            self.max_goods_brand = max(self.max_goods_brand,v_brandsn)
            self.max_goods_id = max(self.max_goods_id,v_id)
        return goods_info

    def get_usr_info(self, df):
        # 建立用户信息的字典
        use_info = {}
        #按行索引数据
        for index, row in df.iterrows():
            usr_id = row['user_id']
            usr_clk = row['clk_turn_rate']
            usr_like = row['like_turn_rate']
            usr_addcart =  row['addcart_turn_rate']
            # 将字符数据转成数字并保存在字典中
            use_info[usr_id] = {'usr_id': usr_id,
                                'clk_turn_rate': usr_clk,
                                'like_turn_rate': usr_like,
                                'addcart_turn_rate': usr_addcart}
            self.max_usr_id = max(self.max_usr_id, usr_id)
            self.max_usr_clk = max(self.max_usr_clk, usr_clk)
            self.max_usr_like = max(self.max_usr_like, usr_like)
            self.max_usr_addcart = max(self.max_usr_addcart, usr_addcart)
        return use_info
    # 得到购买数据
    def get_rating_info(self, df):
        # 将数据保存在字典中并返回
        rating_info = {}
        for index, row in df.iterrows():
            usr_id,goods_id,score = row['user_id'],row['goods_id'],row['score']
            if usr_id not in rating_info.keys():
                rating_info[usr_id] = {goods_id:float(score)}
            else:
                rating_info[usr_id][goods_id] = float(score)
        return rating_info
    # 构建数据集
    def get_dataset(self, usr_info, rating_info, goods_info):
        trainset = []
        for usr_id in rating_info.keys():
            usr_ratings = rating_info[usr_id]
            for goods_id in usr_ratings:
                trainset.append({'usr_info': usr_info[usr_id],
                                 'goods_info': goods_info[goods_id],
                                 'scores': usr_ratings[goods_id]})
        return trainset
    
    def load_data(self, dataset=None, mode='train'):

        # 定义数据迭代Batch大小
        BATCHSIZE = 256

        data_length = len(dataset)
        index_list = list(range(data_length))
        # 定义数据迭代加载器
        def data_generator():
            # 训练模式下，打乱训练数据
            if mode == 'train':
                random.shuffle(index_list)
            # 声明每个特征的列表
            usr_id_list,usr_clk_list,usr_like_list,usr_addcart_list = [], [], [], []
            goods_id_list,goods_cat_list,goods_brand_list = [], [], []
            score_list = []
            # 索引遍历输入数据集
            for idx, i in enumerate(index_list):
                # 获得特征数据保存到对应特征列表中
                usr_id_list.append(dataset[i]['usr_info']['usr_id'])
                usr_clk_list.append(dataset[i]['usr_info']['clk_turn_rate'])
                usr_like_list.append(dataset[i]['usr_info']['like_turn_rate'])
                usr_addcart_list.append(dataset[i]['usr_info']['addcart_turn_rate'])

                goods_id_list.append(dataset[i]['goods_info']['goods_id'])
                goods_cat_list.append(dataset[i]['goods_info']['category'])
                goods_brand_list.append(dataset[i]['goods_info']['brand'])

                score_list.append(int(dataset[i]['scores']))
                # 如果读取的数据量达到当前的batch大小，就返回当前批次
                if len(usr_id_list)==BATCHSIZE:
                    # 转换列表数据为数组形式，reshape到固定形状
                    usr_id_arr = np.array(usr_id_list)
                    usr_clk_arr =  np.array(usr_clk_list)
                    usr_like_arr = np.array(usr_like_list)
                    usr_addcart_arr =  np.array(usr_addcart_list)

                    goods_id_arr = np.array(goods_id_list)
                    goods_cat_arr =np.array(goods_cat_list)
                    goods_brand_arr = np.array(goods_brand_list)

                    scores_arr = np.reshape(np.array(score_list), [-1, 1]).astype(np.float32)

                    # 放回当前批次数据
                    yield [usr_id_arr, usr_clk_arr, usr_like_arr, usr_addcart_arr], \
                           [goods_id_arr, goods_cat_arr, goods_brand_arr], scores_arr

                    # 清空数据
                    usr_id_list, usr_clk_list, usr_like_list, usr_addcart_list = [], [], [], []
                    goods_id_list, goods_cat_list, goods_brand_list, score_list = [], [], [], []
        return data_generator


In [21]:
# 声明数据读取类
dataset = GoodsLen(user_data,train_goods,score)
# 定义数据读取器
train_loader = dataset.load_data(dataset=dataset.train_dataset, mode='train')
# 迭代的读取数据， Batchsize = 256
for idx, data in enumerate(train_loader()):
    usr, mov, score = data
    print("打印用户ID，点击转换率，收藏转换率，加购转换率的维度：")
    for v in usr:
        print(v.shape)
    print("打印商品ID，类别，品牌数据的维度：")
    for v in mov:
        print(v.shape)
    break

KeyboardInterrupt: 

In [19]:
import random
import numpy as np

import paddle
from paddle.nn import Linear, Embedding
import paddle.nn.functional as F
import math

class Model(paddle.nn.Layer):
    def __init__(self, user_data,train_goods,score,fc_sizes):
        super(Model, self).__init__()
        self.fc_sizes=fc_sizes
        # 获取数据集的信息，并构建训练和验证集的数据迭代器
        Dataset = GoodsLen(user_data,train_goods,score)
        self.Dataset = Dataset
        self.trainset = self.Dataset.train_dataset
        self.valset = self.Dataset.valid_dataset
        self.train_loader = self.Dataset.load_data(dataset=self.trainset, mode='train')
        self.valid_loader = self.Dataset.load_data(dataset=self.valset, mode='valid')

        """ define network layer for embedding usr info """
        USR_ID_NUM = Dataset.max_usr_id + 1
        # 对用户ID做映射，并紧接着一个Linear层
        self.usr_emb = Embedding(num_embeddings=USR_ID_NUM, embedding_dim=64, sparse=False)
        self.usr_fc = Linear(in_features=64, out_features=64)
        
        # 对用户点击转换率信息做映射，并紧接着一个Linear层
        USR_CLK_DICT_SIZE = Dataset.max_usr_clk+1
        self.usr_clk_emb = Embedding(num_embeddings=USR_CLK_DICT_SIZE, embedding_dim=32)
        self.usr_clk_fc = Linear(in_features=32, out_features=32)
        
        # 对用户收藏转换率信息做映射，并紧接着一个Linear层
        USR_LIKE_DICT_SIZE = Dataset.max_usr_like+1
        self.usr_like_emb = Embedding(num_embeddings=USR_LIKE_DICT_SIZE, embedding_dim=32)
        self.usr_like_fc = Linear(in_features=32, out_features=32)
        
        # 对用户加购转换率信息做映射，并紧接着一个Linear层
        USR_ADDCART_DICT_SIZE = Dataset.max_usr_addcart+1
        self.usr_addcart_emb = Embedding(num_embeddings=USR_ADDCART_DICT_SIZE, embedding_dim=32)
        self.usr_addcart_fc = Linear(in_features=32, out_features=32)
        
        # 新建一个Linear层，用于整合用户数据信息
        self.usr_combined = Linear(in_features=160, out_features=256)
        
        """ define network layer for embedding usr info """
        # 对商品ID信息做映射，并紧接着一个Linear层
        GOODS_DICT_SIZE = Dataset.max_goods_id + 1
        self.goods_emb = Embedding(num_embeddings=GOODS_DICT_SIZE, embedding_dim=64)
        self.goods_fc = Linear(in_features=64, out_features=64)
        
        # 对类别做映射
        CATEGORY_DICT_SIZE = Dataset.max_goods_cat + 1
        self.goods_cat_emb = Embedding(num_embeddings=CATEGORY_DICT_SIZE, embedding_dim=64, sparse=False)
        self.goods_cat_fc = Linear(in_features=64, out_features=64)
        
        # 对brand做映射
        GOODS_BRAND_DICT_SIZE = Dataset.max_goods_brand + 1
        self.goods_brand_emb = Embedding(num_embeddings=GOODS_BRAND_DICT_SIZE, embedding_dim=64, sparse=False)
        self.goods_brand_fc = Linear(in_features=64, out_features=64)
        # 新建一个FC层，用于整合电影特征
        self.goods_concat_embed = Linear(in_features=192, out_features=256)

        user_sizes = [256] + self.fc_sizes
        acts = ["relu" for _ in range(len(self.fc_sizes))]
        self._user_layers = []
        for i in range(len(self.fc_sizes)):
            linear = Linear(
                in_features=user_sizes[i],
                out_features=user_sizes[i + 1],
                weight_attr=paddle.ParamAttr(
                    initializer=paddle.nn.initializer.Normal(
                        std=1.0 / math.sqrt(user_sizes[i]))))
            self.add_sublayer('linear_user_%d' % i, linear)
            self._user_layers.append(linear)
            if acts[i] == 'relu':
                act =  paddle.nn.ReLU()
                self.add_sublayer('user_act_%d' % i, act)
                self._user_layers.append(act)
        
        #商品特征和用户特征使用了不同的全连接层，不共享参数
        goods_sizes = [256] + self.fc_sizes
        acts = ["relu" for _ in range(len(self.fc_sizes))]
        self._goods_layers = []
        for i in range(len(self.fc_sizes)):
            linear = Linear(
                in_features=goods_sizes[i],
                out_features=goods_sizes[i + 1],
                weight_attr=paddle.ParamAttr(
                    initializer=paddle.nn.initializer.Normal(
                        std=1.0 / math.sqrt(goods_sizes[i]))))
            self.add_sublayer('linear_goods_%d' % i, linear)
            self._goods_layers.append(linear)
            if acts[i] == 'relu':
                act = paddle.nn.ReLU()
                self.add_sublayer('movie_act_%d' % i, act)
                self._goods_layers.append(act)
        
    # 定义计算用户特征的前向运算过程
    def get_usr_feat(self, usr_var):
        """ get usr features"""
        # 获取到用户数据
        usr_id, usr_clk, usr_like, usr_addcart= usr_var
        # 将用户的ID数据经过embedding和Linear计算，得到的特征保存在feats_collect中
        feats_collect = []
        usr_id = self.usr_emb(usr_id)
        usr_id = self.usr_fc(usr_id)
        usr_id = F.relu(usr_id)
        feats_collect.append(usr_id)
        
        # 计算用户的点击转换率特征，并保存在feats_collect中
        usr_clk = self.usr_clk_emb(usr_clk)
        usr_clk = self.usr_clk_fc(usr_clk)
        usr_clk = F.relu(usr_clk)
        feats_collect.append(usr_clk)

        # 计算用户的收藏转换率特征，并保存在feats_collect中
        usr_like = self.usr_like_emb(usr_like)
        usr_like = self.usr_like_fc(usr_like)
        usr_like = F.relu(usr_like)
        feats_collect.append(usr_like)
        # 计算用户的加购转换率特征，并保存在feats_collect中
        usr_addcart = self.usr_addcart_emb(usr_addcart)
        usr_addcart = self.usr_addcart_fc(usr_addcart)
        usr_addcart = F.relu(usr_addcart)
        feats_collect.append(usr_addcart)
    
        # 将用户的特征级联，并通过Linear层得到最终的用户特征
        usr_feat = paddle.concat(feats_collect, axis=1)
        user_features = F.tanh(self.usr_combined(usr_feat))
        #通过3层全链接层，获得用于计算相似度的用户特征和电影特征
        for n_layer in self._user_layers:
            user_features = n_layer(user_features)

        return user_features

        # 定义电影特征的前向计算过程
    def get_goods_feat(self, goods_var):
        """ get movie features"""
        # 获得电影数据
        goods_id, goods_cat, goods_brand = goods_var
        feats_collect = []
        # 获得batchsize的大小
        batch_size = goods_id.shape[0]
        # 计算电影ID的特征，并存在feats_collect中
        goods_id = self.goods_emb(goods_id)
        goods_id = self.goods_fc(goods_id)
        goods_id = F.relu(goods_id)
        feats_collect.append(goods_id)
        
        # 计算电影种类的特征映射，对多个种类的特征求和得到最终特征
        goods_cat = self.goods_cat_emb(goods_cat)
        goods_cat = self.goods_cat_fc(goods_cat)
        goods_cat = F.relu(goods_cat)
        feats_collect.append(goods_cat)

        # 计算电影名字的特征映射，对特征映射使用卷积计算最终的特征
        goods_brand = self.goods_brand_emb(goods_brand)
        goods_brand = self.goods_brand_fc(goods_brand)
        goods_brand = F.relu(goods_brand)
        feats_collect.append(goods_brand)
            
        # 使用一个全连接层，整合所有电影特征，映射为一个200维的特征向量
        goods_feat = paddle.concat(feats_collect, axis=1)
        goods_features = F.tanh(self.goods_concat_embed(goods_feat))

        for n_layer in self._goods_layers:
            goods_features = n_layer(goods_features)

        return goods_features
    
    # 定义个性化推荐算法的前向计算
    def forward(self, usr_var, goods_var):
        # 计算用户特征和电影特征
        user_feat = self.get_usr_feat(usr_var)
        goods_feat = self.get_goods_feat(goods_var)
       
        # 根据计算的特征计算相似度
        sim = F.common.cosine_similarity(user_feat, goods_feat).reshape([-1, 1])
        return user_feat, goods_feat, sim

In [20]:
def train(model):
    # 配置训练参数
    lr = 0.001
    Epoches = 10
    paddle.set_device('cpu') 

    # 启动训练
    model.train()
    # 获得数据读取器
    data_loader = model.train_loader
    # 使用adam优化器，学习率使用0.01
    opt = paddle.optimizer.Adam(learning_rate=lr, parameters=model.parameters())
    
    for epoch in range(0, Epoches):
        for idx, data in enumerate(data_loader()):
            # 获得数据，并转为tensor格式
            usr, mov, score = data
            usr_v = [paddle.to_tensor(var) for var in usr]
            mov_v = [paddle.to_tensor(var) for var in mov]
            scores_label = paddle.to_tensor(score)
            # 计算出算法的前向计算结果
            _, _, scores_predict = model(usr_v, mov_v)
            # 计算loss
            loss = F.square_error_cost(scores_predict, scores_label)
            avg_loss = paddle.mean(loss)

            if idx % 500 == 0:
                print("epoch: {}, batch_id: {}, loss is: {}".format(epoch, idx, avg_loss.numpy()))
                
            # 损失函数下降，并清除梯度
            avg_loss.backward()
            opt.step()
            opt.clear_grad()

        # 每个epoch 保存一次模型
        paddle.save(model.state_dict(), './checkpoint/epoch'+str(epoch)+'.pdparams')

In [133]:
# 启动训练
fc_sizes=[256, 128, 64]
model = Model(user_data,train_goods,score,fc_sizes)
train(model)

##Total dataset instances:  4455772
##GoodsLens dataset information: 
usr num: 19066
goods num: 1531526
epoch: 0, batch_id: 0, loss is: [0.09458251]
epoch: 0, batch_id: 500, loss is: [0.]
epoch: 0, batch_id: 1000, loss is: [0.]
epoch: 0, batch_id: 1500, loss is: [0.]
epoch: 0, batch_id: 2000, loss is: [0.]
epoch: 0, batch_id: 2500, loss is: [0.]
epoch: 0, batch_id: 3000, loss is: [0.]
epoch: 0, batch_id: 3500, loss is: [0.]
epoch: 0, batch_id: 4000, loss is: [0.]
epoch: 0, batch_id: 4500, loss is: [0.]
epoch: 0, batch_id: 5000, loss is: [0.]
epoch: 0, batch_id: 5500, loss is: [0.]
epoch: 0, batch_id: 6000, loss is: [0.]
epoch: 0, batch_id: 6500, loss is: [0.]
epoch: 0, batch_id: 7000, loss is: [0.]
epoch: 0, batch_id: 7500, loss is: [0.]
epoch: 0, batch_id: 8000, loss is: [0.]
epoch: 0, batch_id: 8500, loss is: [0.]
epoch: 0, batch_id: 9000, loss is: [0.]
epoch: 0, batch_id: 9500, loss is: [0.]
epoch: 0, batch_id: 10000, loss is: [0.]
epoch: 0, batch_id: 10500, loss is: [0.]
epoch: 0, 

KeyboardInterrupt: 

In [21]:
# 加载第三方库Pickle，用来保存Python数据到本地
import pickle
# 定义特征保存函数
def get_usr_goods_features(model, params_file_path):
    paddle.set_device('cpu') 
    usr_pkl = {}
    goods_pkl = {}
    
    # 定义将list中每个元素转成tensor的函数
    def list2tensor(inputs, shape):
        inputs = np.reshape(np.array(inputs).astype(np.int64), shape)
        return paddle.to_tensor(inputs)

    # 加载模型参数到模型中，设置为验证模式eval（）
    model_state_dict = paddle.load(params_file_path)
    model.load_dict(model_state_dict)
    model.eval()
    # 获得整个数据集的数据
    dataset = model.Dataset.dataset

    for i in range(len(dataset)):
        # 获得用户数据，电影数据，评分数据  
        # 本案例只转换所有在样本中出现过的user和movie，实际中可以使用业务系统中的全量数据
        usr_info, goods_info, score = dataset[i]['usr_info'], dataset[i]['goods_info'],dataset[i]['scores']
        usrid = str(usr_info['usr_id'])
        goodsid = str(goods_info['goods_id'])

        # 获得用户数据，计算得到用户特征，保存在usr_pkl字典中
        if usrid not in usr_pkl.keys():
            usr_id_v = list2tensor(usr_info['usr_id'], [1])
            usr_clk_v = list2tensor(usr_info['clk_turn_rate'], [1])
            usr_like_v = list2tensor(usr_info['like_turn_rate'], [1])
            usr_addcart_v = list2tensor(usr_info['addcart_turn_rate'], [1])

            usr_in = [usr_id_v, usr_clk_v, usr_like_v, usr_addcart_v]
            usr_feat = model.get_usr_feat(usr_in)

            usr_pkl[usrid] = usr_feat.numpy()
        
        # 获得电影数据，计算得到电影特征，保存在mov_pkl字典中
        if goodsid not in goods_pkl.keys():
            goods_id_v = list2tensor(goods_info['goods_id'], [1])
            goods_cat_v = list2tensor(goods_info['category'], [1])
            goods_brand_v = list2tensor(goods_info['brand'], [1])

            goods_in = [goods_id_v, goods_cat_v, goods_brand_v]
            goods_feat = model.get_goods_feat(goods_in)

            goods_pkl[goodsid] = goods_feat.numpy()
    


    print(len(goods_pkl.keys()))
    # 保存特征到本地
    pickle.dump(usr_pkl, open('./usr_feat.pkl', 'wb'))
    pickle.dump(goods_pkl, open('./goods_feat.pkl', 'wb'))
    print("usr / goods features saved!!!")




In [142]:
param_path = "./checkpoint/epoch0.pdparams"
get_usr_goods_features(model, param_path)        

1292774
usr / goods features saved!!!


In [22]:
# 定义根据用户兴趣推荐电影
import paddle
def recommend_goods_for_usr(usr_id, top_k, pick_num, usr_feats, goods_feats, testa_goods):
    assert pick_num <= top_k
    usr_feat = usr_feats[str(usr_id)]
    cos_sims = []

    # with dygraph.guard():
    paddle.disable_static()
    # 索引电影特征，计算和输入用户ID的特征的相似度
    for idx, key in enumerate(goods_feats.keys()):
        goods_feat = goods_feats[key]
        usr_feat = paddle.to_tensor(usr_feat)
        goods_feat = paddle.to_tensor(goods_feat)
        # 计算余弦相似度
        sim = paddle.nn.functional.common.cosine_similarity(usr_feat, goods_feat)
        
        cos_sims.append(sim.numpy()[0])
    # 对相似度排序
    index = np.argsort(cos_sims)[-top_k:]

    goods_info = {}
    # 读取电影文件里的数据，根据电影ID索引到电影信息
    for index1,item in testa_goods.iterrows():
        goods_id = item['goods_id']
        goods_info[str(goods_id)] = goods_id
            
    # print("当前的用户是：")
    # print("usr_id:", usr_id)
    # print("推荐可能喜欢的商品是：")
    res = []
    
    # 加入随机选择因素，确保每次推荐的都不一样
    while len(res) < pick_num:
        val = np.random.choice(len(index), 1)[0]
        idx = index[val]
        goods_id = list(goods_feats.keys())[idx]
        if goods_id not in res:
            res.append(goods_id)

    # for id in res:
    #     print("goods_id:", id, goods_info[str(id)])
    return res


In [151]:
# 读取商品和用户的特征
usr_feats = pickle.load(open('usr_feat.pkl', 'rb'))
goods_feats = pickle.load(open('goods_feat.pkl', 'rb'))
top_k, pick_num = 10, 3
usr_id = 2
recommend_goods_for_usr(usr_id, top_k, pick_num, usr_feats, goods_feats, train_goods)

当前的用户是：
usr_id: 2
推荐可能喜欢的电影是：
goods_id: 126540 126540
goods_id: 137636 137636
goods_id: 141115 141115


In [23]:
import pickle
data = {'user_id':[],'goods_id':[]}
usr_feats = pickle.load(open('usr_feat.pkl', 'rb'))
goods_feats = pickle.load(open('goods_feat.pkl', 'rb'))
i = 0
for index,item in testa_user.iterrows():
    i+=1
    print(i)
    if i==50:
        break
    user_id = item['user_id']
    top_k, pick_num = 5, 3
    res = recommend_goods_for_usr(user_id, top_k, pick_num, usr_feats,goods_feats, train_goods)
    data['user_id'].extend([user_id,user_id,user_id])
    data['goods_id'].extend(res)

data['user_id'] = user_encode.inverse_transform(data['user_id'])
data['goods_id'] = goods_encode.inverse_transform(data['goods_id'])
df =  pd.DataFrame(data)
df.tocsv('u2i.csv',index=False)

1


W0530 22:32:22.164213   231 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.2, Runtime API Version: 11.2
W0530 22:32:22.167605   231 gpu_resources.cc:91] device: 0, cuDNN Version: 8.2.


2


KeyboardInterrupt: 